# Figure 6

## Imports

In [ ]:
import dolfin
import matplotlib.pyplot as plt
import numpy
import sympy

import dolfin_mech                    as dmech
import micro_poro_structure_generator as gen

## Defining geometry and material parameters

In [ ]:
seeds_filename = "Fig6-seeds.dat"
mesh_filebasename = "Fig6-mesh"

domain_y = 1.
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.092

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = 0.,
    remove_seeds = True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])
vertices = numpy.array([[xmin, ymin],
                        [xmax, ymin],
                        [xmax, ymax],
                        [xmin, ymax]])
bbox = [xmin, xmax, ymin, ymax]

V0 = (xmax - xmin)*(ymax - ymin)

In [ ]:
Es = 1.
nus = 0.499
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

## Defining kinematics

In [ ]:
alpha_x, alpha_y, alpha_xy, alpha_yx = sympy.symbols('alpha_x alpha_y alpha_xy alpha_yx')
epsilon_x, epsilon_y, epsilon_xy = sympy.symbols('epsilon_x epsilon_y epsilon_xy')
epsilon = sympy.symbols('epsilon')
alpha_x = epsilon + 1
alpha_y = epsilon + 1

C00, C01, C10, C11 = sympy.symbols('C00 C01 C10 C11')
C_CM = sympy.Matrix([[C00, C01],
                     [C10, C11]])
I_C_CM = sympy.trace(C_CM) 
II_C_CM = (sympy.trace(C_CM)**2 - sympy.trace(C_CM**2))/2 
III_C_CM = sympy.det(C_CM) 
J_CM = sympy.sqrt(III_C_CM)
p = sympy.symbols('p')

F = sympy.Matrix(
        [[alpha_x,    0   ],\
         [   0   , alpha_y]])
J = F.det()
C = F.T * F
E = (C - sympy.eye(2))/2

## Computing linearized model response

In [ ]:
homogenization_problem = dmech.HomogenizationProblem(
    dim=2,
    mesh=mesh,
    mat_params=mat_params["parameters"],
    vertices=vertices,
    vol=V0,
    bbox=bbox)
lmbda_tilde, mu_tilde = homogenization_problem.get_lambda_and_mu()
kappa_tilde = homogenization_problem.get_kappa()

sigma_lin = lmbda_tilde * (2*epsilon) + 2 * mu_tilde * epsilon

## Computing macroscopic model response

In [ ]:
W_skel = (lmbda_tilde/4) * (J_CM**2 - 1 - 2 * sympy.ln(J_CM)) + (mu_tilde/2) * (I_C_CM - 2 - 2 * sympy.ln(J_CM))
Sigma_CM = 2*sympy.diff(W_skel, C_CM) - p * J_CM * C_CM.inv()

Sigma_macro = Sigma_CM.subs(list(zip(C_CM, C)))
sigma_macro = F * Sigma_macro * F.T / J

## Macro-micro model comparision 

## $\sigma - E$

In [ ]:
qois_filename = "Fig6-qois.dat"
res_basename = "Fig6"

load_params = {}
load_params["pf"] = 0.0
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0
load_params["U_bar_00"] = 0.5
load_params["U_bar_11"] = 0.5

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.01, "dt_max":0.1},
    res_basename=res_basename,
    verbose=1)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_micro = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
sigma_micro = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_micro = [sigma_/Es for sigma_ in sigma_micro]

In [ ]:
lam_lin = sympy.lambdify(epsilon, sigma_lin, modules=['numpy'])
lam = sympy.lambdify(epsilon, sigma_macro.subs(p, 0)[0,0], modules=['numpy'])

epsilon_vals = numpy.linspace(0, 1, 100)

sigma_macro_vals = lam(epsilon_vals)
sigma_vals_lin = lam_lin(epsilon_vals)

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x, E_y~()$', fontsize=16)
plt.ylabel(r'$\sigma/E_s~()$', fontsize=16)
plt.plot(epsilon_vals, sigma_vals_lin, '#005824')
plt.plot(eps_micro, sigma_micro, '#084594')
plt.plot(epsilon_vals, sigma_macro_vals, '#99000D')
plt.legend(['Linear model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Microscopic model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Macroscopic model '+ r'$\bar{\sigma}_{xx},~\bar{\sigma}_{yy}$'])
plt.xlim(0, 0.5)
plt.ylim(0, 0.15)
# plt.savefig('macro-micro_small.pdf',bbox_inches='tight')
plt.show()

## $\sigma - p_f$

In [ ]:
load_params = {}
load_params["pf"] = 0.5
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0
load_params["U_bar_00"] = 0.0
load_params["U_bar_11"] = 0.0

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.01, "dt_max":0.1},
    res_basename=res_basename,
    verbose=1)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
p_lst = qois_vals[:, qois_name_list.index("p_f") - 1]
sigma_xx_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_micro = [sigma_/Es for sigma_ in sigma_micro]

In [ ]:
sigma_lin_p = -p

lam_in = sympy.lambdify(p, sigma_lin_p, modules=['numpy'])
lam = sympy.lambdify(p, sigma_macro[0,0].subs(epsilon, 0), modules=['numpy'])

p_vals = numpy.linspace(0, 1, 100)
sigma_xx_lam_lin = lam_in(p_vals)
sigma_xx_lam = lam(p_vals)

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$p_f~(kPa)$', fontsize=16)
plt.ylabel(r'$\sigma/{E_s}~()$', fontsize=16)
plt.plot(p_vals, sigma_xx_lam_lin, '#005824')
plt.plot(p_lst, sigma_xx_lst, '#084594')
plt.plot(p_vals, sigma_xx_lam, '#99000D')
plt.legend(['Linear model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Microscopic model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Macroscopic model '+ r'$\bar{\sigma}_{xx},~\bar{\sigma}_{yy}$'])

plt.xlim(0, 0.5)
plt.ylim(-0.5, 0)

plt.show()
# plt.savefig('pf_sigma_incompressible.pdf', bbox_inches='tight')